In [1]:
import numpy as np
from groundlight import Groundlight
import glob
import cv2
import matplotlib.pyplot as plt
import matplotlib
import os
import PIL
from PIL import Image
from io import BytesIO
np.random.seed(0)

In [2]:
GROUNDLIGHT_API_TOKEN = ''

In [3]:
def sweepLocalize(det, img, verbose = False):
    
    imgDims = np.array([img.shape[1],img.shape[0]])
    
    #Recursive Base Case, i.e. too many splits
    if imgDims[0] < 20 or imgDims[1] < 20:
        return None
    
    kernelDims = (imgDims*2/3).astype(int)
    sweepDims = (imgDims/4).astype(int)
    colSweeps, rowSweeps = np.round(imgDims/sweepDims).astype(int) - 1
    
    if verbose:
        fig, ax = plt.subplots(rowSweeps, colSweeps, figsize = (16,12))
    
    sliceConfidences = np.zeros((rowSweeps, colSweeps))
    sliceDict = {}
    
    for col in range(colSweeps):
        for row in range(rowSweeps):
            pxStart = sweepDims * np.array([col, row])
            pxEnd = pxStart + kernelDims
            
            sliceDict[(row, col)] = (pxStart, pxEnd)
            sliceImgMat = img[pxStart[1]:pxEnd[1],pxStart[0]:pxEnd[0],:]
            
            sliceImgPIL = Image.fromarray(cv2.cvtColor(sliceImgMat, cv2.COLOR_BGR2RGB))
            byte_io = BytesIO()
            
            sliceImgPIL.save(byte_io, 'jpeg')
            jpg_buffer = byte_io.getvalue()
            byte_io.close()
            
            image_query = gl.submit_image_query(detector_id=det.id, image = jpg_buffer)
            
            sliceLabel = image_query.result.label
            sliceConf = image_query.result.confidence
            
            if sliceLabel == 'PASS':
                if verbose:
                    ax[row, col].set_title('cube found, conf: '+ str(np.round(sliceConf,2)))
                    ax[row, col].imshow(sliceImgMat)
                sliceConfidences[row, col] = sliceConf
            else:
                if verbose:
                    ax[row, col].imshow(cv2.cvtColor(sliceImgMat,cv2.COLOR_BGR2GRAY), cmap = 'ocean')
                sliceConfidences[row, col] = 0
                
    max1dInd = np.argmax(sliceConfidences)
    max2dInd = np.unravel_index(max1dInd, sliceConfidences.shape)
    
    bestConf = sliceConfidences[max2dInd]
    if bestConf < 0.65:
        return None
    
    if verbose:
        ax[max2dInd[0], max2dInd[1]].set_title('CUBE FOUND, BEST CONF: ' + str(np.round(bestConf,2)), fontsize = 15, color = 'lime')
        plt.show()
    
    bestPxStart, bestPxEnd = sliceDict[max2dInd]
    bestSliceImgMat = img[bestPxStart[1]:bestPxEnd[1],bestPxStart[0]:bestPxEnd[0],:]
    
    return (bestPxStart, np.minimum(bestPxEnd, imgDims), sweepLocalize(det, bestSliceImgMat))

In [4]:
def sweepLocalizeColorDepth(det, colorImg, depthImg, verbose = False):
    
    imgDims = np.array([colorImg.shape[1],colorImg.shape[0]])
    
    #Recursive Base Case, i.e. too many splits
    if imgDims[0] < 20 or imgDims[1] < 20:
        return None
    
    kernelDims = (imgDims*2/3).astype(int)
    sweepDims = (imgDims/4).astype(int)
    colSweeps, rowSweeps = np.round(imgDims/sweepDims).astype(int) - 1
    
    if verbose:
        fig, ax = plt.subplots(rowSweeps, colSweeps, figsize = (16,12))
    
    sliceConfidences = np.zeros((rowSweeps, colSweeps))
    sliceDict = {}
    
    for col in range(colSweeps):
        for row in range(rowSweeps):
            pxStart = sweepDims * np.array([col, row])
            pxEnd = pxStart + kernelDims
            
            sliceDict[(row, col)] = (pxStart, pxEnd)
            sliceColorImgMat = colorImg[pxStart[1]:pxEnd[1],pxStart[0]:pxEnd[0],:]
            sliceDepthImgMat = depthImg[pxStart[1]:pxEnd[1],pxStart[0]:pxEnd[0],:]
            
            sliceImgMat = np.concatenate((sliceColorImgMat, sliceDepthImgMat), axis = 1)
            
            sliceImgPIL = Image.fromarray(cv2.cvtColor(sliceImgMat, cv2.COLOR_BGR2RGB))
            byte_io = BytesIO()
            
            sliceImgPIL.save(byte_io, 'jpeg')
            jpg_buffer = byte_io.getvalue()
            byte_io.close()
            
            image_query = gl.submit_image_query(detector_id=det.id, image = jpg_buffer)
            
            sliceLabel = image_query.result.label
            sliceConf = image_query.result.confidence
            
            if sliceLabel == 'PASS':
                if verbose:
                    ax[row, col].set_title('cube found, conf: '+ str(np.round(sliceConf,2)))
                    ax[row, col].imshow(sliceColorImgMat)
                sliceConfidences[row, col] = sliceConf
            else:
                if verbose:
                    ax[row, col].imshow(cv2.cvtColor(sliceColorImgMat,cv2.COLOR_BGR2GRAY), cmap = 'ocean')
                sliceConfidences[row, col] = 0
                
    max1dInd = np.argmax(sliceConfidences)
    max2dInd = np.unravel_index(max1dInd, sliceConfidences.shape)
    
    bestConf = sliceConfidences[max2dInd]
    if bestConf < 0.65:
        return None
    
    if verbose:
        ax[max2dInd[0], max2dInd[1]].set_title('CUBE FOUND, BEST CONF: ' + str(np.round(bestConf,2)), fontsize = 15, color = 'lime')
        plt.show()
    
    bestPxStart, bestPxEnd = sliceDict[max2dInd]
    bestSliceColorImgMat = colorImg[bestPxStart[1]:bestPxEnd[1],bestPxStart[0]:bestPxEnd[0],:]
    bestSliceDepthImgMat = depthImg[bestPxStart[1]:bestPxEnd[1],bestPxStart[0]:bestPxEnd[0],:]
    
    return (bestPxStart, np.minimum(bestPxEnd, imgDims), sweepLocalizeColorDepth(det, bestSliceColorImgMat, bestSliceDepthImgMat, verbose = verbose))

In [5]:
def assemblePxTreeHelper(oldPx, tree):
    newPx = oldPx + tree[0]
    if(tree[2]) is None:
        newPx += ((tree[1] - tree[0])/2).astype(int)
        return newPx
    return assemblePxTreeHelper(newPx, tree[2])

def assemblePxTree(tree):
    return assemblePxTreeHelper(np.array([0,0]), tree)

In [6]:
def plotTreeOnImage(tree, img):
    fig, ax = plt.subplots(1,1, figsize = (10,8))
    ax.imshow(img)
    plotTreeOnImageHelper(ax, tree, np.array([0,0]))
    return fig, ax
                           
def plotTreeOnImageHelper(ax, tree, prevPx):
    
    cmap = matplotlib.cm.get_cmap('rainbow')
    
    if tree[2] is None:
        center = prevPx + ((tree[0] + tree[1])/2).astype(int)
        ax.scatter([center[0]], [center[1]], marker = 'x', s = 60, color = 'black')
        return
    
    color = cmap(np.random.rand())
    ax.scatter([tree[0][0] + prevPx[0]],[tree[0][1]  + prevPx[1]],marker = 'o', s = 30, color = color)
    ax.scatter([tree[1][0]  + prevPx[0]],[tree[1][1]  + prevPx[1]],marker = 'o', s = 30, color = color)
    prevPx += tree[0]
    plotTreeOnImageHelper(ax, tree[2], prevPx)